In [4]:
import os
from dotenv import load_dotenv

os.chdir('/teamspace/studios/this_studio/interview-question-creator')

In [31]:
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

In [10]:
DATA_ROOT = './data'

In [11]:
!pwd

/teamspace/studios/this_studio/interview-question-creator


In [13]:
from langchain.document_loaders import PyPDFDirectoryLoader, PyPDFLoader

file_1 = os.path.join(DATA_ROOT, 'python-1.pdf')
file1_loader = PyPDFLoader(file_1)

file_1_data = file1_loader.load()

In [16]:
file_1_data[20:25]

[Document(metadata={'source': './data/python-1.pdf', 'page': 20}, page_content='Contents xxi\n19.7 defaultdict . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 188\n19.8 Named tuples . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 189\n19.9 Gathering keyword args . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 190\n19.10 Glossary . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 191\n19.11 Exercises . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 192\nA Debugging 193\nA.1 Syntax errors . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 193\nA.2 Runtime errors . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 195\nA.3 Semantic errors . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 198\nB Analysis of Algorithms 201\nB.1 Order of growth . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 202\nB.2 Analysi

In [15]:
# file_1_data
len(file_1_data)

244

In [18]:
file_1_data_str =""
for page in file_1_data:
    file_1_data_str += page.page_content
    
    
type(file_1_data_str)

str

In [19]:
from langchain.text_splitter import TokenTextSplitter # -> this user LLM to making chanks

# this expected string or buffer
spliter = TokenTextSplitter(
    model_name = 'gpt-3.5-turbo',
    chunk_size=1000,
    chunk_overlap=200
)

# spliter.split_documents() for the langchain document type

chunks_docs = spliter.split_text(file_1_data_str)

In [23]:
print(len(chunks_docs))

print(type(chunks_docs))
print(type(chunks_docs[0]))

147
<class 'list'>
<class 'str'>


In [25]:
# we have to conver it to the document type it's recomended way
from langchain.docstore.document import Document

chunks_data = [Document(page_content=t) for t in chunks_docs]
print(type(chunks_data[0]))
print(len(chunks_data))

<class 'langchain_core.documents.base.Document'>
147


In [41]:
from langchain.chat_models import ChatOpenAI
llm_model = ChatOpenAI(
    model='gpt-3.5-turbo',
)

In [34]:
prompt_template = """
You are an expret at creating questions based on python and coding. 
Your goal is to prepare a good programmer for their exam and coding tests for evaluation.
You do this by asking question about the information below:
------
{text}
------
Create questions that will prepare a programmer for their test. Make sure not to loss any important topic or information.

Questions:
"""


In [44]:
refine_template = ("""
You are an expert at creating practice question based on coding. Your goal is to help a programmer prepare for a coding test. 
We have received some practice question to a certain extent, {existing_answer} 
We have option to refine the existing question or add new ones. 
(only if necessary) with some more context below.
----
{text}
----
Given the new context. refine the original question in English.
If the context is not helpful, please provide the original question.
Questions:
"""
)
# here existing_answer would be come from previous output from llm

In [45]:
from langchain.prompts import PromptTemplate

PROMPT_QUESTIONS= PromptTemplate(
    template=prompt_template,
    input_variables =['text']
)

REFINE_PROMPT = PromptTemplate(
    template=refine_template,
    input_variable= ['existing_answer', 'text']
)

In [46]:
from langchain.chains.summarize import load_summarize_chain

question_generation_chain = load_summarize_chain(
    llm = llm_model,
    chain_type='refine',
    verbose=True,
    question_prompt= PROMPT_QUESTIONS,
    refine_prompt= REFINE_PROMPT
)

In [47]:
ques = question_generation_chain.run(chunks_data)



> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

You are an expret at creating questions based on python and coding. 
Your goal is to prepare a good programmer for their exam and coding tests for evaluation.
You do this by asking question about the information below:
------
Think Python
How to Think Like a Computer Scientist
2nd Edition, Version 2.4.0Think Python
How to Think Like a Computer Scientist
2nd Edition, Version 2.4.0
Allen Downey
Green Tea Press
Needham, MassachusettsCopyright © 2015 Allen Downey.
Green Tea Press
9 Washburn Ave
Needham MA 02492
Permission is granted to copy, distribute, and/or modify this document under the terms of the
Creative Commons Attribution-NonCommercial 3.0 Unported License, which is available at http:
//creativecommons.org/licenses/by-nc/3.0/ .
The original form of this book is L ATEX source code. Compiling this L ATEX source has the effect of gen-
erating a device-independent representatio


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

You are an expert at creating practice question based on coding. Your goal is to help a programmer prepare for a coding test. 
We have received some practice question to a certain extent, 1. What licensing terms allow users to copy, modify, and distribute the book "Think Python" by Allen Downey?

2. What were the goals that Allen Downey set when writing his own programming book?

3. What was the original programming language that Allen Downey was teaching before switching to Python?

4. What new features were added in the second edition of "Think Python"?

5. How did Jeff Elkner contribute to the development of "Think Python"?

6. What is the significance of the title "Think Python" in relation to the book's content?

7. How does the structure of "Think Python" differ from traditional programming books, according to Allen Downey?

8. What are the main differences in teaching Python versus Java, as experienc

In [49]:
ques.split('\n')

['1. Explain the concept of algorithm analysis in computer science and its importance. Provide a scenario where algorithm analysis played a crucial role in decision-making.',
 '',
 '2. Enhance the Hand class by creating a method that enables the creation of multiple hands and distributes the appropriate number of cards per hand. This method should return a list of Hands, using keyword arguments for flexible function parameter handling.',
 '',
 '3. How does encoding ranks and suits in a standard playing card object representation aid in comparing cards based on their attributes? Illustrate the implementation of encoding in a Card class using named tuples and inheritance in object-oriented programming.']

In [51]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

embeddings = OpenAIEmbeddings()
vector_store = FAISS.from_documents(chunks_data, embeddings)

/tmp/ipykernel_11432/264093367.py:4: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


In [52]:
questions = ques.split('\n')
questions

['1. Explain the concept of algorithm analysis in computer science and its importance. Provide a scenario where algorithm analysis played a crucial role in decision-making.',
 '',
 '2. Enhance the Hand class by creating a method that enables the creation of multiple hands and distributes the appropriate number of cards per hand. This method should return a list of Hands, using keyword arguments for flexible function parameter handling.',
 '',
 '3. How does encoding ranks and suits in a standard playing card object representation aid in comparing cards based on their attributes? Illustrate the implementation of encoding in a Card class using named tuples and inheritance in object-oriented programming.']

In [53]:
from langchain.chains import RetrievalQA

answer_generation_chain = RetrievalQA.from_chain_type(
    llm = llm_model,
    chain_type='stuff',
    retriever= vector_store.as_retriever()
)

In [55]:
questions

['1. Explain the concept of algorithm analysis in computer science and its importance. Provide a scenario where algorithm analysis played a crucial role in decision-making.',
 '',
 '2. Enhance the Hand class by creating a method that enables the creation of multiple hands and distributes the appropriate number of cards per hand. This method should return a list of Hands, using keyword arguments for flexible function parameter handling.',
 '',
 '3. How does encoding ranks and suits in a standard playing card object representation aid in comparing cards based on their attributes? Illustrate the implementation of encoding in a Card class using named tuples and inheritance in object-oriented programming.']

In [54]:
for question in questions:
    print("Question: ", question)
    answer = answer_generation_chain.run(question)
    print("Answer: ", answer)
    print("--------------------------------------------------\\n\\n")
    # Save answer to file
    with open("answers.txt", "a") as f:
        f.write("Question: " + question + "\\n")
        f.write("Answer: " + answer + "\\n")
        f.write("--------------------------------------------------\\n\\n")

Question:  1. Explain the concept of algorithm analysis in computer science and its importance. Provide a scenario where algorithm analysis played a crucial role in decision-making.
Answer:  Algorithm analysis in computer science is a field that focuses on studying the performance of algorithms, especially their runtime and space requirements. The goal of algorithm analysis is to predict how different algorithms will perform in order to guide design decisions. It helps in understanding the efficiency of algorithms and making informed choices about which algorithm to use based on the problem at hand.

An example of algorithm analysis playing a crucial role in decision-making is in the selection of sorting algorithms. During the 2008 United States Presidential Campaign, candidate Barack Obama was asked to provide the most efficient way to sort a million 32-bit integers during a visit to Google. This scenario highlighted the importance of understanding algorithm efficiency in real-world a